In this code, we generate samples using the JulianSchwingerModel package, and use the samples to implement RG schemes. The schemes are a lot more involved here though.

Initialize parameters!

In [ ]:
using JulianSchwingerModel
import TensorOperations, NPZ

nx = 16
nt = 16
kappa = 0.23              # hopping parameter
mass = 0.5
beta = 0.5
quenched = false

# Set up HMC parameters
tau =  1                   # leapfrog integration time
integrationsteps = 300
thermalizationiter = 10     # total number of accepted thermalization
hmciter = 200               # total number of accepted measurements

# Initilize lattice
lattice = Lattice(nx, nt, mass, beta, quenched)

HMC update to generate valid samples

In [ ]:
hmcparam = HMCParam(tau, integrationsteps, thermalizationiter, hmciter)

# Print basic lattice information
print_lattice(lattice)
n = 1
samples = []
# Thermalize lattice
for i in 1:n
    HMCWilson_continuous_update!(lattice, hmcparam)
    push!(samples,lattice)
end

This part of the code could not be imported from the package. We define the class Pseudofermions, which stores data for Pseudofermion vectors.

In [ ]:
struct PseudoFermion
    pf::FlatField
    function PseudoFermion(lattice::Lattice, g5Dslash_vec::Any)
        pf = FlatField(undef, 2*lattice.ntot)
        Dm1pf = FlatField(undef, 2*lattice.ntot)
        for i in 1:lattice.ntot
            # Sample D^{-1}\phi according to normal distribution
            Dm1pf[dirac_comp1(i)] = (gauss() + im*gauss())/sqrt(2)
            Dm1pf[dirac_comp2(i)] = (gauss() + im*gauss())/sqrt(2)
        end
        pf = gamma5mul(g5Dslash_vec(Dm1pf, lattice, lattice.mass))
        new(pf)
    end
end

In [ ]:
pf = PseudoFermion(lattice, gamma5_Dslash_wilson_vector)

This is where we define all the functions needed to compute the score function, which a lot more involed.

In [ ]:
"""
S_gauge = sum_i beta(1 - Re U_{plaq})
Caluate the gauge action contribution at lattice
site i.  The constant beta * 1 is ignored in action.
"""
function SG(i::Int64, lattice::Lattice, beta::Float64)
    return - beta *
           cos(lattice.anglex[i] + lattice.anglet[lattice.rightx[i]] -
               lattice.anglex[lattice.upt[i]] - lattice.anglet[i])
end

"""
Gauge force for U(x=i, mu=1)
"""
function dSG1(i::Int64, lattice::Lattice, beta::Float64)
    return - beta * (
           sin(lattice.anglex[lattice.downt[i]] +
               lattice.anglet[lattice.rightx[lattice.downt[i]]] -
               lattice.anglex[i] - lattice.anglet[lattice.downt[i]]) -
           sin(lattice.anglex[i] + lattice.anglet[lattice.rightx[i]] -
               lattice.anglex[lattice.upt[i]] - lattice.anglet[i])
            )
end

"""
Gauge force for U(x=i, mu=2)
"""
function dSG2(i::Int64, lattice::Lattice, beta::Float64)
    return beta * (
            sin(lattice.anglex[lattice.leftx[i]] + lattice.anglet[i] -
                lattice.anglex[lattice.leftx[lattice.upt[i]]] -
                lattice.anglet[lattice.leftx[i]]) -
            sin(lattice.anglex[i] + lattice.anglet[lattice.rightx[i]] -
                lattice.anglex[lattice.upt[i]] - lattice.anglet[i])
            )
end

"""
Internal function used in pforce1 and pforce2. Calculate the left-hand-side of
dot product that is independent of mu.

CG inversion takes place here. Q is returned by gamma5_Dslash_linearmap so
we dont have to reconstruct it everytime we call
"""

function pforce_common(Q::Any, pf::PseudoFermion, lattice::Lattice, mass::Float64)
    # First invert pf
    psi = minres_Q(Q, lattice, mass, gamma5mul(pf.pf))
    Dm1_gamma5_psi = minres_Q(Q, lattice, mass, psi)
    return gamma5mul(Dm1_gamma5_psi), psi
end

#Implements M^-1 psi
function f(Q::Any,psi::FlatField, lattice::Lattice, mass::Float64)
    gamma5_psi = gamma5mul(psi)
    Qinv_gamma5_psi = minres_Q(Q, lattice, mass, gamma5_psi)
    return Qinv_gamma5_psi
end

#Implements M' psi for theta_1
function g_theta1(i::Int64, Q::Any, psi::FlatField, lattice::Lattice)
    link1 = lattice.linkx[i]
    right1_i = lattice.rightx[i]
    psi_new = copy(psi)
    psi_new[dirac_comp1(i)] = -im/2*(link1*(psi[dirac_comp1(right1_i)]-psi[dirac_comp2(right1_i)])-conj(link1)*(psi[dirac_comp1(i)]+psi[dirac_comp2(i)]))
    psi_new[dirac_comp2(i)] = -im/2*(link1*(-psi[dirac_comp1(right1_i)]+psi[dirac_comp2(right1_i)])-conj(link1)*(psi[dirac_comp1(i)]+psi[dirac_comp2(i)]))
    return psi_new
end

#Implements M' psi for theta_2
# Implements M' psi for theta_2 (First derivative)
function g_theta2(i::Int64, Q::Any, psi::FlatField, lattice::Lattice)
    link2 = lattice.linkt[i]
    right2_i = lattice.upt[i]
    # Apply antiperiodic boundary condition
    bterm = (lattice.corr_indx[right2_i][2] == 1) ? -1 : 1
    # Create new vector instead of modifying `psi`
    psi_new = copy(psi)
    psi_new[dirac_comp1(i)] = -im/2 * (link2 * (bterm * psi[dirac_comp1(right2_i)] - psi[dirac_comp2(right2_i)]) - conj(link2) * (psi[dirac_comp1(i)] + psi[dirac_comp2(i)]))
    psi_new[dirac_comp2(i)] = -im/2 * (link2 * (-bterm * psi[dirac_comp1(right2_i)] + psi[dirac_comp2(right2_i)]) - conj(link2) * (psi[dirac_comp1(i)] + psi[dirac_comp2(i)]))
    return psi_new
end

#Implements M" psi for theta_1
function h_theta1(i::Int64, Q::Any, psi::FlatField, lattice::Lattice)
    link1 = lattice.linkx[i]
    right1_i = lattice.rightx[i]
    psi_new = copy(psi)
    psi_new[dirac_comp1(i)] = 1/2*(link1*(psi[dirac_comp1(right1_i)]-psi[dirac_comp2(right1_i)])+conj(link1)*(psi[dirac_comp1(i)]+psi[dirac_comp2(i)]))
    psi_new[dirac_comp2(i)] = 1/2*(link1*(-psi[dirac_comp1(right1_i)]+psi[dirac_comp2(right1_i)])+conj(link1)*(psi[dirac_comp1(i)]+psi[dirac_comp2(i)]))
    return psi_new
end

#Implements M" psi for theta_2
function h_theta2(i::Int64, Q::Any, psi::FlatField, lattice::Lattice)
    link2 = lattice.linkt[i]
    right2_i = lattice.upt[i]
    # Apply antiperiodic boundary condition
    bterm = (lattice.corr_indx[right2_i][2] == 1) ? -1 : 1
    # Create new vector instead of modifying `psi`
    psi_new = copy(psi)
    psi_new[dirac_comp1(i)] = 1/2 * (link2 * (bterm * psi[dirac_comp1(right2_i)] - psi[dirac_comp2(right2_i)]) + conj(link2) * (psi[dirac_comp1(i)] + psi[dirac_comp2(i)]))
    psi_new[dirac_comp2(i)] = 1/2 * (link2 * (-bterm * psi[dirac_comp1(right2_i)] + psi[dirac_comp2(right2_i)]) + conj(link2) * (psi[dirac_comp1(i)] + psi[dirac_comp2(i)]))
    return psi_new
end

"""
Fermion forces at site i, mu=1. Using equation (2.81) of Luscher 2010, ``Computational
Strategies in Lattice QCD``. lhs is the output from pforce_common.
"""
function pforce1_theta(i::Int64, pf::PseudoFermion, lattice::Lattice, psi::FlatField, lhs::FlatField)
    left1_i = lattice.leftx[i]
    right1_i = lattice.rightx[i]
    link1 = lattice.linkx[i]

    # Chain rule factors: d/dθ of conj(link1) = -im*conj(link1) and d/dθ of link1 = im*link1
    prod1 = conj(lhs[dirac_comp1(i)]-lhs[dirac_comp2(i)])*(im)*link1*(psi[dirac_comp1(right1_i)]-psi[dirac_comp2(right1_i)])

    prod2 = -conj(lhs[dirac_comp1(i)]+lhs[dirac_comp2(i)])*(im)*conj(link1)*(psi[dirac_comp1(i)]+psi[dirac_comp2(i)])

    return real(prod1 + prod2)
end


"""
Fermion forces at site i, mu=2. Using equation (2.81) of Luscher 2010, ``Computational
Strategies in Lattice QCD``. It is important to have boundary condition consistent with
the Dslash operator. lhs is the output from pforce_common.
psi = D^{-1}phi where phi is pf field.
"""
function pforce2_theta(i::Int64, pf::PseudoFermion, lattice::Lattice, psi::FlatField,
                         lhs::FlatField)
    # Neighbor in the time direction
    right2_i = lattice.upt[i]
    # Implement antiperiodic BC in time: flip sign at the boundary
    if lattice.corr_indx[right2_i][2] == 1
        bterm = -1
    else
        bterm = 1
    end
    psi_right2_1i = bterm * psi[dirac_comp1(right2_i)]
    psi_right2_2i = bterm * psi[dirac_comp2(right2_i)]
    lhs_right2_1i = bterm * lhs[dirac_comp1(right2_i)]
    lhs_right2_2i = bterm * lhs[dirac_comp2(right2_i)]
    link2 = lattice.linkt[i]

    # First term in the dot product
    prod1 = conj(lhs[dirac_comp1(i)]-(im)*lhs[dirac_comp2(i)])*(im)*link2*(psi[dirac_comp1(right2_i)]+(im)*psi[dirac_comp2(right2_i)])

    # Second term in the dot product
    prod2 = -conj(lhs[dirac_comp1(i)]+(im)*lhs[dirac_comp2(i)])*(im)*link2*(psi[dirac_comp1(right2_i)]-(im)*psi[dirac_comp2(right2_i)])
    # Return derivative with respect to theta_2
    return real(prod1 + prod2)
end

function ddSG1(i::Int64, lattice::Lattice, beta::Float64)
    # S_G(i) = -β cos(θ_x(i) + θ_t(rightx[i]) - θ_x(upt[i]) - θ_t(i))
    # => d²S_G/dθ² ≈ β cos(θ_x(i) + θ_t(rightx[i]) - θ_x(upt[i]) - θ_t(i))
    return beta * (
           cos(lattice.anglex[lattice.downt[i]] +
               lattice.anglet[lattice.rightx[lattice.downt[i]]] -
               lattice.anglex[i] - lattice.anglet[lattice.downt[i]]) +
           cos(lattice.anglex[i] + lattice.anglet[lattice.rightx[i]] -
               lattice.anglex[lattice.upt[i]] - lattice.anglet[i])
            )
end

"""
Compute the second derivative of the gauge action with respect to the temporal angle
at site i. (Diagonal approximation.)
"""
function ddSG2(i::Int64, lattice::Lattice, beta::Float64)
    # For simplicity, we use the same combination of angles as in d2SG1.
    return beta * (
        cos(lattice.anglex[lattice.leftx[i]] + lattice.anglet[i] -
            lattice.anglex[lattice.leftx[lattice.upt[i]]] -
            lattice.anglet[lattice.leftx[i]]) +
        cos(lattice.anglex[i] + lattice.anglet[lattice.rightx[i]] -
            lattice.anglex[lattice.upt[i]] - lattice.anglet[i])
        )
end

Calculate the term $\chi_1 = \psi^{\dagger}M'^{\dagger}(M^{\dagger})^{-1}M^{-1}M'\psi$. We first calculate $X = f\circ g\circ \psi = M^{-1}M' \psi$. The answer should be $X^{\dagger}X$.

In [ ]:
function dd_pforce1(i::Int64, psi::FlatField, lattice::Lattice, mass::Float64)
    #This is the first term
    Q = gamma5_Dslash_linearmap(lattice, mass)
    g_psi = g_theta1(i,Q,psi,lattice)
    f_g_psi = f(Q,g_psi,lattice,mass)
    term1 = conj(f_g_psi[dirac_comp1(i)])*f_g_psi[dirac_comp1(i)] + conj(f_g_psi[dirac_comp2(i)])*f_g_psi[dirac_comp2(i)]
    
    #Now the second term 
    gamma5_psi = gamma5mul(psi)
    Minv_gamma5_psi = f(Q,gamma5_psi,lattice,mass)
    Mprime_Minv_gamma5_psi = g_theta1(i,Q,Minv_gamma5_psi,lattice)
    gamma5_Mprime_Minv_gamma5_psi = gamma5mul(Mprime_Minv_gamma5_psi)
    chi = gamma5_Mprime_Minv_gamma5_psi
    term2 = conj(chi[dirac_comp1(i)])*f_g_psi[dirac_comp1(i)] + conj(chi[dirac_comp2(i)])*f_g_psi[dirac_comp2(i)]

    #Now the third term
    chi1 = gamma5mul(minres_Q(Q, lattice, mass, psi))
    chi2 = h_theta1(i,Q,psi,lattice)
    term3 = conj(chi1[dirac_comp1(i)])*chi2[dirac_comp1(i)] + conj(chi1[dirac_comp2(i)])*chi2[dirac_comp2(i)]

    return real(-term1 - 2*term2 + term3)
end

function dd_pforce2(i::Int64, psi::FlatField, lattice::Lattice, mass::Float64)
    #This is the first term
    Q = gamma5_Dslash_linearmap(lattice, mass)
    g_psi = g_theta2(i,Q,psi,lattice)
    f_g_psi = f(Q,g_psi,lattice,mass)
    term1 = conj(f_g_psi[dirac_comp1(i)])*f_g_psi[dirac_comp1(i)] + conj(f_g_psi[dirac_comp2(i)])*f_g_psi[dirac_comp2(i)]
    
    #Now the second term 
    gamma5_psi = gamma5mul(psi)
    Minv_gamma5_psi = f(Q,gamma5_psi,lattice,mass)
    Mprime_Minv_gamma5_psi = g_theta2(i,Q,Minv_gamma5_psi,lattice)
    gamma5_Mprime_Minv_gamma5_psi = gamma5mul(Mprime_Minv_gamma5_psi)
    chi = gamma5_Mprime_Minv_gamma5_psi
    term2 = conj(chi[dirac_comp1(i)])*f_g_psi[dirac_comp1(i)] + conj(chi[dirac_comp2(i)])*f_g_psi[dirac_comp2(i)]

    #Now the third term
    chi1 = gamma5mul(minres_Q(Q, lattice, mass, psi))
    chi2 = h_theta2(i,Q,psi,lattice)
    term3 = conj(chi1[dirac_comp1(i)])*chi2[dirac_comp1(i)] + conj(chi1[dirac_comp2(i)])*chi2[dirac_comp2(i)]

    return real(-term1 - 2*term2 + term3)
end

We compute the score function using all the helper functions defined previously.

In [ ]:
using LinearAlgebra

function total_score_function(lattice::Lattice, pf::PseudoFermion, mass::Float64, beta::Float64)
    ntot = lattice.ntot

    # 1. Build Q and obtain common fermion fields via pforce_common:
    Q = gamma5_Dslash_linearmap(lattice, mass)
    lhs, psi = pforce_common(Q, pf, lattice, mass)
    # 2. Compute gauge score components (first derivative)
    # For spatial gauge angles:
    psi_theta_x = [ ( dSG1(i, lattice,beta) + pforce1_theta(i, pf, lattice, psi, lhs) ) for i in 1:ntot ]
    #psi_theta_x = [ -( dSG1(i, lattice)) for i in 1:ntot ]
    
    # For temporal gauge angles:
    psi_theta_t = [ ( dSG2(i, lattice,beta) + pforce2_theta(i, pf, lattice, psi, lhs) ) for i in 1:ntot ]
    #psi_theta_t = [ -( dSG2(i, lattice)) for i in 1:ntot ]
    # 3. Compute diagonal second derivatives (by hand) for gauge fields:
    div_theta_x = sum( - ( ddSG1(i, lattice,beta) + dd_pforce1(i, psi, lattice,mass) ) for i in 1:ntot )
    #div_theta_x = sum( - ( ddSG1(i, lattice)) for i in 1:ntot )
    div_theta_t = sum( - ( ddSG2(i, lattice,beta) + dd_pforce2(i,psi,lattice,mass) ) for i in 1:ntot )
    #div_theta_t = sum( - ( ddSG2(i, lattice)) for i in 1:ntot )
    # Total gauge contribution to the score:
    score_gauge = (div_theta_x + div_theta_t +
                  0.5 * ( sum(x^2 for x in psi_theta_x) + sum(x^2 for x in psi_theta_t) ))

    # 4. Compute pseudofermion part.
    # Instead of Q \ (gamma5mul(pf.pf)), we use the iterative solver minres_Q.
    #psi_pf = gamma5mul(tmp_pf)
    inprod_pf = sum(abs2, lhs)
    #inprod_psi = sum(abs2,psi)
    Q_inv_2  =  inv(Matrix(Q*Q))
    trace_Q2_inv = real(tr(Q_inv_2))
    score_pf = -4*trace_Q2_inv - 2 * inprod_pf
    # 5. Total score is the sum of gauge and pseudofermion contributions.
    return (score_gauge +  score_pf)
end


In [ ]:
using LinearAlgebra

function total_score_function_new(lattice::Lattice, pf::PseudoFermion, mass::Float64, beta::Float64)
    ntot = lattice.ntot

    # 1. Build Q and obtain common fermion fields via pforce_common:
    Q = gamma5_Dslash_linearmap(lattice, mass)
    lhs, psi = pforce_common(Q, pf, lattice, mass)
    # 2. Compute gauge score components (first derivative)
    # For spatial gauge angles:
    psi_theta_x = [ ( dSG1(i, lattice,beta) + pforce1_theta(i, pf, lattice, psi, lhs) ) for i in 1:ntot ]
    #psi_theta_x = [ -( dSG1(i, lattice)) for i in 1:ntot ]
    
    # For temporal gauge angles:
    psi_theta_t = [ ( dSG2(i, lattice,beta) + pforce2_theta(i, pf, lattice, psi, lhs) ) for i in 1:ntot ]
    #psi_theta_t = [ -( dSG2(i, lattice)) for i in 1:ntot ]
    # 3. Compute diagonal second derivatives (by hand) for gauge fields:
    div_theta_x = sum( - ( ddSG1(i, lattice,beta) + dd_pforce1(i, psi, lattice,mass) ) for i in 1:ntot )
    #div_theta_x = sum( - ( ddSG1(i, lattice)) for i in 1:ntot )
    div_theta_t = sum( - ( ddSG2(i, lattice,beta) + dd_pforce2(i,psi,lattice,mass) ) for i in 1:ntot )
    #div_theta_t = sum( - ( ddSG2(i, lattice)) for i in 1:ntot )
    # Total gauge contribution to the score:
    score_gauge = (div_theta_x + div_theta_t +
                  0.5 * ( sum(x^2 for x in psi_theta_x) + sum(x^2 for x in psi_theta_t) ))

    # 4. Compute pseudofermion part.
    newscore_f = -2 * lattice.ntot * (mass + 2)
    # 5. Total score is the sum of gauge and pseudofermion contributions.
    return (score_gauge +  newscore_f)
end

In [ ]:
function full_sample_score(lattice::Lattice, pfmatrix::Any, mass::Float64, beta::Float64)
    n = length(pfmatrix)
    score = 0.0
    for p in pfmatrix
        score += total_score_function(lattice, p, mass,beta)
    end
    return score/n
end

We learn the values of $\beta$ and $m$ here.

In [ ]:
using Optim

# Define a loss function that accepts parameters as a vector.
function loss_function_optim(x::Vector{Float64})
    β = x[1]
    m = x[2]
    # Compute and return the total score matching loss.
    return total_score_function_new(lattice, pf, m,β)
end

# Initial guess for parameters: [β, m]
initial_params = [0.2, 0.1]

# Run the optimization using the Nelder-Mead/LBFGS method.
result = optimize(loss_function_optim, initial_params, NelderMead())

# Extract the optimized parameters.
opt_params = Optim.minimizer(result)
β_opt, m_opt = opt_params[1], opt_params[2]

println("Optimized beta: ", β_opt)
println("Optimized mass: ", m_opt)
println("Actual beta: ", beta)
println("Actual mass: ", mass)


This is where we find the errors for each pseudofermion matrix, however this is very computationally expensive, and we can only verify that the errors decrease, but not their scaling data.

In [ ]:
sample_sizes = [10*2^i for i in 0:4]
b_errors = []
m_errors = []

for size in sample_sizes
    pfm = []
    for i in 1:size
        push!(pfm,PseudoFermion(lattice, gamma5_Dslash_wilson_vector))
    end

    function loss_function_optim(x::Vector{Float64})
        β = x[1]
        m= x[2]
        # Compute and return the total score matching loss.
        return full_sample_score(lattice, pfm, m,β)
    end
    
    # Initial guess for parameters: [β, m]
    initial_params = [0.2, 0.1]
    
    # Run the optimization using the Nelder-Mead/LBFGS method.
    result = optimize(loss_function_optim, initial_params,NelderMead())
    
    # Extract the optimized parameters.
    opt_params = Optim.minimizer(result)
    β_opt, m_opt = opt_params[1], opt_params[2] 
    b_err = abs(β_opt - beta)
    m_err = abs(m_opt - mass)
    push!(b_errors,b_err)
    push!(m_errors,m_err)
end


In [ ]:
f(x) = total_score_function(lattice,pf, x[1], x[2])

# Set an initial guess and bounds (assume a and b are positive)
initial_x = [0.5, 0.5]
lower = [0.1, 0.0]
upper = [5.0, 5.0]

# Create an instance of ParticleSwarm with the desired options
ps = ParticleSwarm(lower=lower, upper=upper)

# Run the global optimizer using GalacticOptim.optimize
result = optimize(f, initial_x, ps)

# Extract the minimizer and the minimum objective value
x_opt = result.minimizer
f_opt = result.minimum

println("Final objective value: ", f_opt)
println("Estimated parameters: a = ", x_opt[1], ", b = ", x_opt[2])

println(x_opt[1], x_opt[2])

We define the coarse graining scheme for the links of the lattice, and return the lattice object with the coarse-grained links.

In [ ]:
function coarse_grain_lattice(lattice::Lattice)
    nx, nt = lattice.nx, lattice.nt
    if nx % 2 != 0 || nt % 2 != 0
        error("Lattice dimensions must be even for RG step.")
    end
    nx_new, nt_new = nx ÷ 2, nt ÷ 2
    N_new = nx_new * nt_new

    anglex_new = zeros(Float64, N_new)
    anglet_new = zeros(Float64, N_new)

    function idx_fine(i, j)
        return mod1(j, nt) + (mod1(i, nx) - 1) * nt
    end
    function idx_coarse(i, j)
        return mod1(j, nt_new) + (mod1(i, nx_new) - 1) * nt_new
    end

    for i in 1:nx_new
        for j in 1:nt_new
            idx_c = idx_coarse(i, j)

            # Fine lattice anchor site
            i_f, j_f = 2i - 1, 2j - 1

            # Horizontal link: θ_x + θ_x (i+1, j)
            θ1 = lattice.anglex[idx_fine(i_f, j_f)]
            θ2 = lattice.anglex[idx_fine(i_f + 1, j_f)]
            anglex_new[idx_c] = θ1 + θ2
            # Vertical link: θ_t + θ_t (i, j+1)
            ϕ1 = lattice.anglet[idx_fine(i_f, j_f)]
            ϕ2 = lattice.anglet[idx_fine(i_f, j_f + 1)]
            anglet_new[idx_c] =ϕ1 + ϕ2
        end
    end

    # Construct new coarse lattice (placeholders for mass/beta)
    coarse_lattice = Lattice(nx_new, nt_new, lattice.mass, lattice.beta, lattice.quenched)

    # Overwrite angles and links (reshape needed)
    coarse_lattice.anglex .= reshape(anglex_new, size(coarse_lattice.anglex))
    coarse_lattice.anglet .= reshape(anglet_new, size(coarse_lattice.anglet))

    # Recompute links from coarse angles
    coarse_lattice.linkx .= exp.(im .* coarse_lattice.anglex)
    coarse_lattice.linkt .= exp.(im .* coarse_lattice.anglet)

    return coarse_lattice
end


Here, we coarse-grain the pseudofermion. We first get the coarse grained lattice, then find the right pseudofermions by coarse-graining the old pseudofermions, and then define functions implementing full RG trajectories.

In [ ]:
using Statistics
# This function performs a full RG trajectory for multiple steps:
# - Repeatedly coarse-grains the lattice and pseudofermion field
# - Learns new beta and mass at each scale
# - Returns the full list of couplings across scales

using Optim

# Custom constructor to bypass operator assumptions when constructing pseudofermion
function PseudoFermion_from_field(lattice::Lattice, pf_data::Vector{ComplexF64})
    pf_obj = PseudoFermion(lattice, gamma5_Dslash_wilson_vector)  # uses dummy field
    pf_obj.pf .= pf_data  # overwrite with real data
    return pf_obj
end

function coarse_grain_pf(pf::PseudoFermion, lattice::Lattice)
    nx, nt = lattice.nx, lattice.nt
    if (nx % 2 != 0) || (nt % 2 != 0)
        error("Lattice size must be even for coarse graining.")
    end

    nx2, nt2       = div(nx,2), div(nt,2)
    coarse_lattice = coarse_grain_lattice(lattice)
    N2             = nx2*nt2
    coarse_pf_vec  = Vector{ComplexF64}(undef, 2*N2)   # Dirac comps interleaved

    # --- helper -------------------------------------------------------------
    idx(i,j,dims)   = mod1(j,dims[2]) + (mod1(i,dims[1])-1)*dims[2]
    complex_median(v) = complex(median(real.(v)), median(imag.(v)))
    # ------------------------------------------------------------------------

    for i in 1:nx2, j in 1:nt2
        coarse_site = mod1(j,nt2) + (mod1(i,nx2)-1)*nt2
        c1, c2      = dirac_comp1(coarse_site), dirac_comp2(coarse_site)

        block_sites = (
            idx(2i-1,2j-1,(nx,nt)), idx(2i  ,2j-1,(nx,nt)),
            idx(2i-1,2j  ,(nx,nt)), idx(2i  ,2j  ,(nx,nt))
        )

        d1_vals = ComplexF64[];  d2_vals = ComplexF64[]
        for site in block_sites
            d1, d2 = dirac_comp1(site), dirac_comp2(site)
            push!(d1_vals, pf.pf[d1]);  push!(d2_vals, pf.pf[d2])
        end

        coarse_pf_vec[c1] = complex_median(d1_vals)
        coarse_pf_vec[c2] = complex_median(d2_vals)
    end

    # ------------- Variance-matching normalisation -------------------- #
    mean_sq_fine   = mean(abs2, pf.pf)
    mean_sq_coarse = mean(abs2, coarse_pf_vec)
    Zpf            = max(mean_sq_coarse / mean_sq_fine, eps())  # guard divide-by-0
    coarse_pf_vec ./=sqrt(Zpf)                                 # rescale
    # ----------------------------------------------------------------------- #

    return PseudoFermion_from_field(coarse_lattice, coarse_pf_vec), coarse_lattice
end

const βmin = -10.0      #  forbid β ≤ 0
const βmax = 10.0
const mmin = -15.0
const mmax =  15.0

function rg_step(lattice::Lattice,
                 pf::PseudoFermion,
                 β_guess::Float64,
                 m_guess::Float64)

    # 1) block the gauge field
    coarse_lat = coarse_grain_lattice(lattice)

    # 2) block the pseudo-fermion
    pf_coarse, _ = coarse_grain_pf(pf, lattice)

    # 3) loss(x)  (x = [β, m])
    loss_fn(x) = total_score_function(coarse_lat, pf_coarse, x[2], x[1])

    # 4) bounds
    lower = [βmin, mmin]
    upper = [βmax, mmax]

    # 5) Optimise with bound-aware Nelder–Mead
    loss_fn(x) = total_score_function(coarse_lat, pf_coarse, x[2], x[1])

    x0     = [clamp(β_guess, βmin, βmax), clamp(m_guess, mmin, mmax)]
    lower  = [βmin, mmin]
    upper  = [βmax, mmax]
    opts   = Optim.Options(iterations = 2000)
    result = optimize(x -> total_score_function(coarse_lat, pf_coarse, x[2], x[1]),
                  lower, upper, x0, Fminbox(NelderMead()), opts)
    β_opt, m_opt = Optim.minimizer(result)
    println("RG step: β = $(β_opt),   m = $(m_opt)   (loss = $(result.minimum))")

    return coarse_lat, pf_coarse, β_opt, m_opt
end

function rg_trajectory(lattice::Lattice, pf::PseudoFermion, beta0::Float64, mass0::Float64, nsteps::Int)
    betas = [beta0]
    masses = [mass0]
    lattices = [lattice]
    pfs = [pf]

    current_lattice = lattice
    current_pf = pf
    beta_guess = beta0
    mass_guess = mass0

    for step in 1:nsteps
        println("\n--- RG Step $(step) ---")
        new_lattice, new_pf, new_beta, new_mass = rg_step(current_lattice, current_pf, beta_guess, mass_guess)

        push!(betas, new_beta)
        push!(masses, new_mass)
        push!(lattices, new_lattice)
        push!(pfs, new_pf)

        current_lattice = new_lattice
        current_pf = new_pf
        beta_guess = new_beta
        mass_guess = new_mass
    end

    return betas, masses, lattices, pfs
end


Here we finally learn RG trajectrories for parameters. However, the parameter estimation becomes less reliable for smaller lattices, so the procedure is to
1. Generate samples for large lattice size
2. RG under lattice size is order 8x8
3. Generate samples for previous RG parameters with larger lattice again
4. Learn and repeat.

In [ ]:
betas,masses,_1,_2 = rg_trajectory(lattice,pf,lattice.beta,lattice.mass,2)